# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## Scraping wiki table to get the first DataFrame

### import libraries in order to scrape the table

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('libraries imported')

libraries imported


### read HTML as list and convert it into DF

In [2]:
raw_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
raw_df = pd.DataFrame(raw_list[0],columns =['Postal Code','Borough','Neighborhood'])


### drop not assigned boroughs

In [3]:
tor_df = raw_df.loc[raw_df['Borough'] != 'Not assigned'].reset_index(drop=True)

### check the dataframe through "head" and "shape" methods

In [4]:
print('This is what the dataframe looks like now:')
tor_df.head()

This is what the dataframe looks like now:


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
print('This is the shape of the Toronto Dataframe: ',tor_df.shape)

This is the shape of the Toronto Dataframe:  (103, 3)


## Get Latitude and Logitude for each of the Postal Codes

### borrowed code :-|

In [6]:
import geocoder # import geocoder
print('Geocoder imported')

Geocoder imported


In [7]:
# initialize your variable to None
lat_lng_coords = None

In [ ]:
# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('postal_code'))
  lat_lng_coords = g.latlng

In [9]:
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

TypeError: 'NoneType' object is not subscriptable